https://github.com/huggingface/transformers/blob/main/examples/pytorch/summarization/README.md

https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/bart#transformers.BartForConditionalGeneration

https://discuss.huggingface.co/t/train-bart-for-conditional-generation-e-g-summarization/1904

| BART Model: | Description |
| :----------- | :-----------: | 
| bart.base |	            BART model with 6 encoder and decoder layers |
| bart.large |	            BART model with 12 encoder and decoder layers |
| bart.large.mnli |	        bart.large finetuned on MNLI |
| bart.large.cnn |	        bart.large finetuned on CNN-DM |
| bart.large.xsum |         bart.large finetuned on Xsum |

In [55]:
import pandas as pd
import numpy as np

In [56]:
df = pd.read_csv("./Data/BART_PRED.csv")
df1 = pd.read_csv("./Data/BART_PRED_1.csv") 

In [57]:
df[df["BART_Pred"] == ""].index

Index([], dtype='int64')

In [58]:
df1[df1["BART_Pred"] == ""].index

Index([], dtype='int64')

In [59]:
le1 = df[df["BART_Pred"].isna()].index
le1

Index([11986, 15907, 29747, 40129, 46210], dtype='int64')

In [60]:
le2 = df1[df1["BART_Pred"].isna()].index
le2

Index([11986, 15907, 29747, 40129, 46210], dtype='int64')

In [61]:
def all2list(samp_li):
    return samp_li.replace("\'", "").strip('][').replace(',', '').split(" ")

In [62]:
df["document"] = df["document"].map(all2list)

In [63]:
df["leng"] = df["document"].apply(lambda x: len(x))

In [64]:
le = df[df["leng"] <= 3].index
le

Index([11986, 15907, 22710, 29747, 40129, 46210], dtype='int64')

In [67]:
try:
    if le1 == le:
        print("They are the same")
    else:
        print("They are not the same")
except:
    print("error occured, they arent the same")

error occured, they arent the same


In [68]:
df[df["BART_Pred"].isna()]

,Unnamed: 0,document,summary,docString,sumString,BART_Pred,leng
11986,2164402,"[stick, hand, pork]","['expensive', 'pork', 'south', 'china', 'attra...",stick hand pork,expensive pork south china attract robber,NaN,3
15907,2458749,"[dying, excuse, nixon]","['nixon', 'may', 'dead', 'republican', 'still'...",dying excuse nixon,nixon may dead republican still vie blessing,NaN,3
29747,2769662,"[handful, surprise, mr]","['bernie', 'mac', 'connects', 'mr']",handful surprise mr,bernie mac connects mr,NaN,3
40129,2550165,"[unk, back, nothing]","['unk', 'back', 'making', 'noise']",unk back nothing,unk back making noise,NaN,3
46210,2606734,"[enough, say, love]","['love', 'letter', 'straight', 'heart']",enough say love,love letter straight heart,NaN,3


In [69]:
df[df["leng"] <= 3]

,Unnamed: 0,document,summary,docString,sumString,BART_Pred,leng
11986,2164402,"[stick, hand, pork]","['expensive', 'pork', 'south', 'china', 'attra...",stick hand pork,expensive pork south china attract robber,NaN,3
15907,2458749,"[dying, excuse, nixon]","['nixon', 'may', 'dead', 'republican', 'still'...",dying excuse nixon,nixon may dead republican still vie blessing,NaN,3
22710,2690100,"[enough, hospital, anymore]","['odyssey', 'child', 'hospital', 'lead', 'space']",enough hospital anymore,odyssey child hospital lead space,enough,3
29747,2769662,"[handful, surprise, mr]","['bernie', 'mac', 'connects', 'mr']",handful surprise mr,bernie mac connects mr,NaN,3
40129,2550165,"[unk, back, nothing]","['unk', 'back', 'making', 'noise']",unk back nothing,unk back making noise,NaN,3
46210,2606734,"[enough, say, love]","['love', 'letter', 'straight', 'heart']",enough say love,love letter straight heart,NaN,3


In [ ]:
def runBart(df):

    # Empty lists for predictions and performance timestamps
    predictions = []
    times = []

    # For the number of rows in the given dataframe
    for i in range(len(df)):
        # Create a start timestamp
        start = time.perf_counter()

        # Create a document instance using the row's entry for the stringified document
        doc = df.iloc[i]["docString"]

        # Encoding inputs using BART tokenizer 
        inputs = tokenizer.encode(doc, return_tensors='pt', max_length=1024, truncation=True).to(device)

        # Generate vectorized summary using encoded inputs
        summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

        # Decode the summary into a human-readable format
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Append the predicted summary to a list of predictions
        predictions.append(summary)

        # Create an end timestamp
        end = time.perf_counter()

        # Calculate computation speed
        speed = end - start

        # Append computation speed to list
        times.append(speed)

        # If the iteration is a multiple of 1000
        if i % 1000 == 0:
            # Calculate the average computation time per row so far and print
            avg_time = sum(times) / len(times)
            print("Average time per row at", i, "row:", avg_time)

    # Create a new column for the dataframe using the predictions generated and return the modified dataframe
    df["BART_Pred"] = predictions
    return df